# Learn to draw

#### Drawing frontend
https://draw.neurohub.io<br>

#### Flow API
https://flow.neurohub.io<br>


# Import Libraries

In [128]:
import pandas as pd
import torch
import torchvision
from torchvision import transforms
import os
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
import requests
import json
from json import JSONEncoder


In [130]:
class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

# List drawings in the database 

In [129]:
# GET LIST OF DRAWINGS
ids = pd.read_json('https://flow.neurohub.io/list')

In [134]:
ids

,0
0,712fff63-3aa0-4fa1-b3c1-97fd671f0490
1,e031a812-2852-4570-a092-8db5089f2554
2,1b8d9b91-806a-4506-8891-e4fbd84c8613
3,f1a05f72-09e4-4686-8cb5-22f29e6254bf
4,48dcbfe7-c51f-4250-9473-f3378ead7809
...,...
87,88233446-8863-43d0-ab0d-b1486948b5c6
88,2aff9b4e-242b-4666-9334-fcf6adce0d79
89,fc8d014c-7cdc-41b4-8411-cfa71afec1c9
90,5e45ca3e-eefc-49e6-a793-eb9a666e8d4c


# Test simplification of 1 drawing

In [131]:
# // TEST STROKE SIMPLIFICATION 

id='6960a950-8522-43f8-ac04-45a10a1013f8';

points = pd.read_json('https://flow.neurohub.io/drawings/' + id + '/points');

headers = {
  'Content-Type': 'application/json'
}

s=.1
numPoints = 32;
payload = [];
strokeIds = points["strokeId"].unique();

for strokeId in strokeIds:
  strokePoints = points[points["strokeId"] == strokeId];
  x, y = strokePoints["x"].to_numpy(), strokePoints["y"].to_numpy() 

  if x.size>5:      
    coords = np.linspace(0, x.size, x.size)
    k=np.min([x.size,5])
    splx = UnivariateSpline(coords, x, k=k, s=s)
    sply = UnivariateSpline(coords, y, k=k, s=s)
    xs = np.linspace(0, x.size, numPoints)
    ys = np.linspace(0, y.size, numPoints)
    data = {'strokeId':strokeId,
            'xs':splx(xs),
            'ys':sply(ys)
    }
    payload.append(data);

data = json.dumps(payload, cls=NumpyArrayEncoder)
URL = "https://flow.neurohub.io/drawings/" + id + "/simple"

response = requests.request("PUT", URL, headers=headers, data=data)
response

<Response [200]>

# Simplify all drawings in the database 

In [140]:
# // STROKE SIMPLIFICATION 

headers = {
  'Content-Type': 'application/json'
}

s=.1
numPoints = 32;

for id in ids[0]:  
  if not id.endswith("-s"):
    print(id)
    points = pd.read_json('https://flow.neurohub.io/drawings/' + id + '/points');
    strokeIds = points["strokeId"].unique();

    payload = []
    for strokeId in strokeIds:
      strokePoints = points[points["strokeId"] == strokeId];
      x, y = strokePoints["x"].to_numpy(), strokePoints["y"].to_numpy() 

      if x.size>5:      
        coords = np.linspace(0, x.size, x.size)
        k=np.min([x.size,5])
        splx = UnivariateSpline(coords, x, k=k, s=s)
        sply = UnivariateSpline(coords, y, k=k, s=s)
        xs = np.linspace(0, x.size, numPoints)
        ys = np.linspace(0, y.size, numPoints)
        data = {'strokeId':strokeId,
                'xs':splx(xs),
                'ys':sply(ys)
        }
        payload.append(data);

    data = json.dumps(payload, cls=NumpyArrayEncoder)
    URL = "https://flow.neurohub.io/drawings/" + id +"/simple"
    response = requests.request("PUT", URL, headers=headers, data=data)


712fff63-3aa0-4fa1-b3c1-97fd671f0490
e031a812-2852-4570-a092-8db5089f2554
1b8d9b91-806a-4506-8891-e4fbd84c8613
f1a05f72-09e4-4686-8cb5-22f29e6254bf
48dcbfe7-c51f-4250-9473-f3378ead7809
09f211bb-5866-4da6-895f-6e380614620c
e6508dd5-9a97-4485-ba12-cb65331ec574
7c22d5cd-2cce-400c-82a5-4681a1126e80
6960a950-8522-43f8-ac04-45a10a1013f8
e464b0a2-e99a-422b-9411-1709e229945e
6ac93a11-63e7-4a52-8f36-1fc64b63f2c0
d63cf011-c0eb-44b4-b862-67637aec2efb
d0297d4d-33ba-46e6-8f70-d34672270ed5
5a9ac0e7-a5f8-4f4d-adfe-92f2292c59e6
7b7be45c-8e82-40eb-84d7-4e9b5920084d
a2665f22-a98d-4cd3-bfaa-4a97fdd12ec3
fe7e4e09-15ed-447d-972e-83769bb747e4
23041050-253d-43a7-8117-4e691868a6f5
7c381fbe-e5b7-4abb-9be9-0bea80132e21
d8121e2d-26b4-4f5c-8aa2-5584b2868cdf
8cf1bc4b-d152-4340-948c-f10c53e64dd3
4aec00c1-3bfd-4baf-a2a9-cbc0414e923e
9072ecaa-91b1-454c-a150-4e8b0e91314f
3dad4a25-b777-4aa1-960f-5275162c67a8
d20dfa30-48aa-412e-b160-4c9627dc932e
fff6bd5f-247f-4416-962e-4887d64e869d
773bd8e0-d91e-4856-9c4c-80b55263d8ac
f